In [7]:
import supervision as sv
import numpy as np
import cv2
from ultralytics import YOLO
import pandas as pd
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
model_defect_od = YOLO('../terminal_crop_defect_object_detection/best_defect_od_crop.pt')

def callback(x: np.ndarray) -> sv.Detections:
    result = model_defect_od(x, verbose=False)[0]
    return sv.Detections.from_ultralytics(result)

In [3]:
model_terminal_od = YOLO('../terminal_object_detection/best.pt')

In [4]:
df = pd.read_pickle('../data/common_files/df_markup.pkl')
df = df[
    (df['terminal']==1) &
    (df['file_name'].apply(lambda x: 'augm' not in x))
]

In [5]:
sample_images = df['file_name'].sample(60).values

In [ ]:
for sample_image in tqdm(sample_images):
    image = Image.open(sample_image)
    results_terminal_od = model_terminal_od.predict(image, verbose=False)
    try:
        x1, y1, x2, y2 = results_terminal_od[0].boxes.xyxy[0].tolist()
    except:
        continue
    cropped_im = image.crop((x1, y1, x2, y2))
    cropped_im = cv2.cvtColor(np.array(cropped_im), cv2.COLOR_RGB2BGR)
    slicer = sv.InferenceSlicer(callback=callback)
    sliced_detections = slicer(image=cropped_im)
    box_annotator = sv.BoxAnnotator()
    sliced_image = box_annotator.annotate(cropped_im.copy(), 
                                          detections=sliced_detections, 
                                          labels=['body_attrition', 'body_dirt', 'body_scratch', 'key_attrition', 'key_dirt', 'screen_attrition', 'screen_dirt', 'screen_scratch']
                                          )
    im = Image.fromarray(sliced_image)
    im.save('sahi_examples/'+sample_image.split('\\')[-1])
    sv.plot_image(sliced_image)
    plt.show()
clear_output()

Эксперимент неудачный, качество прогнозов очень низкое :(